In [1]:
import pandas as pd

In [2]:
gp2gp_spine_data_files = [
  "s3://prm-gp2gp-data-sandbox-dev/spine-gp2gp-data/Jan-2021.csv.gz",
  "s3://prm-gp2gp-data-sandbox-dev/spine-gp2gp-data/Feb-2021.csv.gz",
  "s3://prm-gp2gp-data-sandbox-dev/spine-gp2gp-data/Mar-2021.csv.gz",
  "s3://prm-gp2gp-data-sandbox-dev/spine-gp2gp-data/Apr-2021.csv.gz",
]

gp2gp_spine = pd.concat((
    pd.read_csv(f, parse_dates=["_time"])
    for f in gp2gp_spine_data_files
))

In [3]:
is_ack = gp2gp_spine["interactionID"] == "urn:nhs:names:services:gp2gp/MCCI_IN010000UK13"
gp2gp_acks = gp2gp_spine[is_ack]

In [4]:
ack_summary = gp2gp_acks.groupby("messageRef").agg(
    errors=pd.NamedAgg(column="jdiEvent", aggfunc=list),
    message_count=pd.NamedAgg(column="jdiEvent", aggfunc=len),
    distinct_error_count=pd.NamedAgg(column="jdiEvent", aggfunc="nunique")
)

In [5]:
ack_summary["message_count"].value_counts()

1      12749328
2         28249
3          2383
4           652
5           233
6           105
7            59
8            30
9            15
10           14
11            8
13            7
12            5
14            5
16            2
21            2
143           1
17            1
18            1
23            1
27            1
28            1
33            1
34            1
525           1
Name: message_count, dtype: int64

In [6]:
ack_summary["distinct_error_count"].value_counts()

1    12765912
2       15131
3          63
Name: distinct_error_count, dtype: int64

In [7]:
has_more_than_one_error_code = ack_summary["distinct_error_count"] > 1
conflicting_acks = ack_summary[has_more_than_one_error_code]

In [8]:
conflicting_codes = conflicting_acks["errors"].apply(lambda e: set(e)).value_counts().to_frame("count")

In [9]:
conflicting_codes

,count
"{NONE, 12}",9566
"{20, NONE}",2117
"{11, NONE}",733
"{15, 12}",719
"{29, NONE}",583
"{15, 11}",417
"{NONE, 10}",261
"{31, NONE}",251
"{23, NONE}",167
"{NONE, 19}",114
